# Energy vs Solar Radiation Correlation Analysis
## Kiểm tra sản lượng năng lượng vs bức xạ mặt trời

Phân tích mối quan hệ giữa sản lượng năng lượng (energy) và bức xạ mặt trời (shortwave_radiation) cho 5 trạm PV.

**Kỳ vọng**: Phải có mối quan hệ đồng biến mạnh (positive correlation) - khi bức xạ tăng thì sản lượng phải tăng.

**Dữ liệu**: 2025-06-01 đến 2025-11-08 (~ 161 ngày, ~19K bản ghi trên mỗi trạm)

**Mục tiêu**:
1. Load dữ liệu từ Silver layer (toàn bộ khoảng thời gian)
2. Kiểm tra correlation Pearson giữa energy và radiation
3. Tìm anomalies (dữ liệu bất thường)
4. Visualize scatter plots và time series

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configure display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

print("✓ Imports successful")

## 2. Load Facility Data and Create Trino Connection

In [ ]:
# Install trino-python-client if needed
import subprocess
import sys

try:
    import trino
except ImportError:
    print("Installing trino-python-client...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "trino", "-q"])
    import trino

from trino.dbapi import connect

# Create connection to Trino (port 8081, not 8080)
conn = connect(
    host='localhost',
    port=8081,
    user='trino',
    catalog='iceberg',
    schema='silver'
)

print("✓ Trino connection established")

## 3. Load Energy and Weather Data from Silver Layer

In [ ]:
# Load energy data (power output)
query_energy = """
SELECT 
    date_hour,
    facility_code,
    facility_name,
    network_region,
    energy_mwh,
    intervals_count,
    is_valid
FROM iceberg.silver.clean_hourly_energy
ORDER BY facility_code, date_hour
"""

energy_df = pd.read_sql(query_energy, conn)
energy_df['date_hour'] = pd.to_datetime(energy_df['date_hour'])

print(f"✓ Loaded {len(energy_df)} energy records")
print(f"  Date range: {energy_df['date_hour'].min()} to {energy_df['date_hour'].max()}")
print(f"  Facilities: {energy_df['facility_code'].unique().tolist()}")

In [ ]:
# Load weather data (solar radiation)
query_weather = """
SELECT 
    date_hour,
    facility_code,
    facility_name,
    shortwave_radiation,
    direct_radiation,
    diffuse_radiation,
    direct_normal_irradiance,
    is_valid
FROM iceberg.silver.clean_hourly_weather
ORDER BY facility_code, date_hour
"""

weather_df = pd.read_sql(query_weather, conn)
weather_df['date_hour'] = pd.to_datetime(weather_df['date_hour'])

print(f"✓ Loaded {len(weather_df)} weather records")
print(f"  Date range: {weather_df['date_hour'].min()} to {weather_df['date_hour'].max()}")
print(f"  Facilities: {weather_df['facility_code'].unique().tolist()}")

## 4. Merge Energy and Weather Data

In [ ]:
# Merge energy and weather on facility_code and date_hour
df = pd.merge(
    energy_df,
    weather_df,
    on=['date_hour', 'facility_code'],
    how='inner',
    suffixes=('_energy', '_weather')
)

print(f"✓ Merged data: {len(df)} records")
print(f"\nData shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

## 5. Data Exploration

In [ ]:
print("="*80)
print("DATA SUMMARY BY FACILITY")
print("="*80)

for facility in sorted(df['facility_code'].unique()):
    fac_data = df[df['facility_code'] == facility]
    print(f"\n{facility} ({fac_data['facility_name_energy'].iloc[0]}) - Region: {fac_data['network_region'].iloc[0]}")
    print(f"  Records: {len(fac_data)}")
    print(f"  Energy (MWh): mean={fac_data['energy_mwh'].mean():.4f}, std={fac_data['energy_mwh'].std():.4f}")
    print(f"  Radiation (W/m²): mean={fac_data['shortwave_radiation'].mean():.2f}, std={fac_data['shortwave_radiation'].std():.2f}")
    print(f"  Missing energy: {fac_data['energy_mwh'].isna().sum()}")
    print(f"  Missing radiation: {fac_data['shortwave_radiation'].isna().sum()}")

## 6. Calculate Correlation Between Energy and Radiation

In [ ]:
print("="*80)
print("PEARSON CORRELATION: Energy vs Shortwave Radiation")
print("="*80)
print("Expected: Strong positive correlation (r > 0.7)")
print()

correlation_results = []

for facility in sorted(df['facility_code'].unique()):
    fac_data = df[df['facility_code'] == facility]
    
    # Remove NaN values for correlation calculation
    clean_data = fac_data[['energy_mwh', 'shortwave_radiation']].dropna()
    
    if len(clean_data) > 1:
        pearson_r, p_value = stats.pearsonr(clean_data['energy_mwh'], clean_data['shortwave_radiation'])
        correlation_results.append({
            'facility_code': facility,
            'facility_name': fac_data['facility_name_energy'].iloc[0],
            'region': fac_data['network_region'].iloc[0],
            'pearson_r': pearson_r,
            'p_value': p_value,
            'significance': 'YES' if p_value < 0.05 else 'NO'
        })
        
        status = '✓' if pearson_r > 0.7 else '⚠' if pearson_r > 0.5 else '✗'
        print(f"{status} {facility}: r={pearson_r:.4f} (p={p_value:.2e}) - {'STRONG' if pearson_r > 0.7 else 'MODERATE' if pearson_r > 0.5 else 'WEAK'} correlation")

corr_df = pd.DataFrame(correlation_results)
print(f"\n{corr_df.to_string(index=False)}")

## 7. Detect Anomalies - Hours with Mismatch

In [ ]:
print("="*80)
print("ANOMALY DETECTION: Energy-Radiation Mismatch")
print("="*80)
print("\nLooking for hours where:")
print("  - High radiation but LOW energy (underperformance)")
print("  - Low radiation but HIGH energy (impossible)")
print()

# Create quartiles for each facility (handling NaN properly)
def safe_qcut(x):
    try:
        result = pd.qcut(x.dropna(), q=4, labels=['Q1_low', 'Q2', 'Q3', 'Q4_high'], duplicates='drop')
        return pd.Series(result, index=x.index)
    except:
        return pd.Series([None] * len(x), index=x.index)

df['radiation_quartile'] = df.groupby('facility_code')['shortwave_radiation'].transform(safe_qcut)
df['energy_quartile'] = df.groupby('facility_code')['energy_mwh'].transform(safe_qcut)

# Find mismatches
anomalies = df[
    ((df['radiation_quartile'] == 'Q4_high') & (df['energy_quartile'] == 'Q1_low')) |  # High rad, low energy
    ((df['radiation_quartile'] == 'Q1_low') & (df['energy_quartile'] == 'Q4_high'))   # Low rad, high energy
]

print(f"\nTotal anomalies found: {len(anomalies)} / {len(df)} ({100*len(anomalies)/len(df):.2f}%)")

if len(anomalies) > 0:
    print(f"\nTop 10 anomalies:")
    anomalies_sorted = anomalies.sort_values('date_hour')
    print(anomalies_sorted[['date_hour', 'facility_code', 'energy_mwh', 'shortwave_radiation', 'radiation_quartile', 'energy_quartile']].head(10).to_string())

## 8. Scatter Plot: Energy vs Radiation by Facility

In [ ]:
# Create scatter plot
fig = px.scatter(
    df,
    x='shortwave_radiation',
    y='energy_mwh',
    color='facility_code',
    hover_data=['facility_code', 'date_hour', 'intervals_count'],
    title='Energy Production vs Solar Radiation by Facility',
    labels={
        'shortwave_radiation': 'Shortwave Radiation (W/m²)',
        'energy_mwh': 'Energy (MWh)'
    },
    height=600,
    width=1000
)

fig.update_traces(marker=dict(size=8, opacity=0.6))
fig.show()

## 9. Time Series: Energy and Radiation Over Time

## 9b. Efficiency Analysis: Energy Output vs Solar Input

Phân tích hiệu suất (efficiency) = Energy / Radiation
- Hiệu suất bình thường: 15-22% cho commercial solar panels
- Nếu nhỏ hơn 10% → Có vấn đề
- Nếu lớn hơn 25% → Không hợp lý (có thể lỗi dữ liệu)

In [ ]:
# Filter: Keep ONLY rows where both Energy and Radiation timestamps match exactly
df_matched = df.dropna(subset=['energy_mwh', 'shortwave_radiation']).copy()

print("="*80)
print("FILTERED DATA: Only Matched Timestamps")
print("="*80)
print(f"\nBefore filtering: {len(df)} records")
print(f"After filtering (removing NaN): {len(df_matched)} records")
print(f"Dropped: {len(df) - len(df_matched)} records (NaN values)")

print(f"\nMatched data by facility:")
for facility in sorted(df_matched['facility_code'].unique()):
    fac_data = df_matched[df_matched['facility_code'] == facility]
    print(f"  {facility}: {len(fac_data)} records")

# Use matched data for all further analysis
df = df_matched

print(f"\n✓ Now using {len(df)} matched records for all analysis below")

In [ ]:
# Calculate efficiency: Energy / Radiation
# Units: Energy in MWh, Radiation in W/m² → need to normalize

# Efficiency ratio (simplified): Energy output per unit radiation
# High radiation hours (> 100 W/m²) only
df_daytime = df[df['shortwave_radiation'] > 100].copy()

# Efficiency = Energy / (Radiation / 1000)  # Normalize radiation
df_daytime['efficiency_ratio'] = df_daytime['energy_mwh'] / (df_daytime['shortwave_radiation'] / 1000)

print("="*80)
print("EFFICIENCY ANALYSIS: Energy Output vs Solar Radiation")
print("="*80)
print("\nFiltering: Daytime hours (Radiation > 100 W/m²)")
print(f"Total records in analysis: {len(df_daytime)}\n")

for facility in sorted(df_daytime['facility_code'].unique()):
    fac_data = df_daytime[df_daytime['facility_code'] == facility]['efficiency_ratio'].dropna()
    
    mean_eff = fac_data.mean()
    std_eff = fac_data.std()
    min_eff = fac_data.min()
    max_eff = fac_data.max()
    
    print(f"{facility}:")
    print(f"  Mean efficiency ratio: {mean_eff:.4f}")
    print(f"  Std deviation: {std_eff:.4f}")
    print(f"  Range: {min_eff:.4f} to {max_eff:.4f}")
    print(f"  Status: ", end="")
    
    if mean_eff < 0.01:
        print("❌ VERY LOW (might have data quality issue)")
    elif mean_eff < 0.02:
        print("⚠️  LOW (possible underperformance)")
    elif mean_eff > 0.03:
        print("⚠️  HIGH (check if realistic)")
    else:
        print("✓ NORMAL")
    print()

# Show hours with lowest efficiency
print("\n" + "="*80)
print("LOWEST EFFICIENCY HOURS (Top 20 underperformers)")
print("="*80)

low_eff = df_daytime.nsmallest(20, 'efficiency_ratio')[
    ['date_hour', 'facility_code', 'energy_mwh', 'shortwave_radiation', 'efficiency_ratio']
].round(4)

print(low_eff.to_string(index=False))

In [ ]:
# Create time series plot for 1 month of data for each facility
facilities = sorted(df['facility_code'].unique())
n_facilities = len(facilities)

# Select 1 month of data (first complete month in dataset)
start_date = df['date_hour'].min()
end_date = start_date + pd.Timedelta(days=30)  # ~1 month

df_month = df[(df['date_hour'] >= start_date) & (df['date_hour'] < end_date)]

print(f"📊 Plotting data from {start_date.date()} to {end_date.date()} (~1 month)")
print(f"   Total records: {len(df_month):,}")

fig = make_subplots(
    rows=n_facilities, cols=1,
    subplot_titles=[f"{fac}" for fac in facilities],
    specs=[[{"secondary_y": True}]] * n_facilities,
    vertical_spacing=0.08
)

for idx, facility in enumerate(facilities, 1):
    fac_data = df_month[df_month['facility_code'] == facility].sort_values('date_hour')
    
    # Add energy line (primary y-axis)
    fig.add_trace(
        go.Scatter(
            x=fac_data['date_hour'],
            y=fac_data['energy_mwh'],
            name=f'{facility} Energy',
            line=dict(color='blue', width=2),
            legendgroup=facility,
            showlegend=True
        ),
        row=idx, col=1, secondary_y=False
    )
    
    # Add radiation line (secondary y-axis)
    fig.add_trace(
        go.Scatter(
            x=fac_data['date_hour'],
            y=fac_data['shortwave_radiation'],
            name=f'{facility} Radiation',
            line=dict(color='orange', width=2, dash='dash'),
            legendgroup=facility,
            showlegend=True
        ),
        row=idx, col=1, secondary_y=True
    )
    
    # Update y-axes labels
    fig.update_yaxes(title_text="Energy (MWh)", row=idx, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Radiation (W/m²)", row=idx, col=1, secondary_y=True)

fig.update_xaxes(title_text="Date", row=n_facilities, col=1)
fig.update_layout(
    title=f'Energy (Blue) vs Shortwave Radiation (Orange) - 1 Month ({start_date.date()} to {end_date.date()})',
    height=250*n_facilities,
    width=1200,
    hovermode='x unified'
)

fig.show()

## 10. Correlation Heatmap

In [ ]:
# Calculate correlation for each facility separately
fig, axes = plt.subplots(1, len(facilities), figsize=(15, 4))

for idx, facility in enumerate(facilities):
    fac_data = df[df['facility_code'] == facility][[
        'energy_mwh', 'shortwave_radiation', 
        'direct_radiation', 'diffuse_radiation', 'direct_normal_irradiance'
    ]].dropna()
    
    corr_matrix = fac_data.corr()
    
    sns.heatmap(
        corr_matrix,
        annot=True,
        fmt='.2f',
        cmap='coolwarm',
        center=0,
        vmin=-1,
        vmax=1,
        ax=axes[idx],
        cbar=False
    )
    axes[idx].set_title(f'{facility}', fontsize=12, fontweight='bold')

plt.suptitle('Correlation Matrix: Energy vs Radiation Metrics', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()